In [1]:
import glob
import os
import numpy as np
import math
from DataProcess import *
from SolutionChecker import *
from LocalSearch import *
import cProfile 
import time

# Test for Input TD

In [2]:
data_path = os.path.dirname(os.path.abspath('__file__')) + "/Instances/inputTD.full"

In [8]:
solution_path = os.path.dirname(os.path.abspath('__file__')) + "/Solutions/inputTD.solution"

In [9]:
print_data(data_path)

-----------------------------
evacuation_tree =  [[1, 48, 8, 3, 4, 5, 6], [2, 30, 5, 3, 4, 5, 6], [3, 33, 3, 2, 5, 6]]
graph =  [[1, 4, 9223372036854775807, 7, 8], [2, 4, 9223372036854775807, 4, 5], [3, 5, 9223372036854775807, 6, 3], [4, 5, 9223372036854775807, 9, 10], [5, 6, 9223372036854775807, 12, 11]]
number of evacuation nodes :  3
List of evac nodes :  [1, 2, 3]
number of nodes =  6
number of edges =  5
-----------------------------


0

In [10]:
print_solution(solution_path)

-----------------------------
A solution is : [[1, 8, 3], [2, 5, 0], [3, 3, 0]]
f_objectif =  37
Nature of solution :  valid
-----------------------------


0

In [11]:
verify_solution(data_path,solution_path)

CREATION TASKS & VERIFY CONSTRAINTS
1 3
2 0
3 0
-----------------------------
CHECK F_OBJECTIF
-----------------------------
End time is :  37
True
-----------------------------
END
-----------------------------


True

In [3]:
EVA_TREE,GRAPH,NB_NODES = read_data(data_path)
NB_EVA_NODES = len(EVA_TREE)
NB_EDGES = len(GRAPH)
LIST_EVA_NODES = [item[0] for item in EVA_TREE]

In [4]:
for i in range(NB_EVA_NODES) : 
    node_id = EVA_TREE[i][0]
    print('Evacuation of (only) node ',node_id,' is ', get_duration(node_id,EVA_TREE,GRAPH),' mins')

Evacuation of (only) node  1  is  34  mins
Evacuation of (only) node  2  is  31  mins
Evacuation of (only) node  3  is  29  mins


In [5]:
print('Borne inferieur = {} mins'.format(get_borne_inf(LIST_EVA_NODES,EVA_TREE,GRAPH)))

Borne inferieur = 34 mins


In [6]:
endtime,init_sol= get_borne_sup(EVA_TREE,GRAPH)
print('Borne superieur = {} mins'.format(endtime))
init_sol

Borne superieur = 40 mins


[[1, 8, 0], [2, 5, 9], [3, 3, 0]]

In [7]:
cp = cProfile.Profile()
cp.enable()
exc_time = time.time()
endtime,solution= LocalSearchRun(init_sol,EVA_TREE,GRAPH,n_iter=10)
exc_time = time.time() - exc_time
create_solution_file('inputTD',solution,endtime,"Local Search Run",exc_time)
cp.disable()
cp.print_stats()

Iteration 0:
[1, 3, 2]  =>  40
Iteration 1:
[2 3 1]  =>  37
Iteration 2:
[2 3 1]  =>  37
Iteration 3:
[2 3 1]  =>  37
Iteration 4:
[2 3 1]  =>  37
Iteration 5:
[2 3 1]  =>  37
         5690 function calls (5660 primitive calls) in 0.132 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <ipython-input-7-0b4812f9ac00>:3(<module>)
        1    0.000    0.000    0.110    0.110 <ipython-input-7-0b4812f9ac00>:4(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-7-0b4812f9ac00>:5(<module>)
        1    0.000    0.000    0.022    0.022 <ipython-input-7-0b4812f9ac00>:6(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-7-0b4812f9ac00>:7(<module>)
      166    0.002    0.000    0.028    0.000 DataProcess.py:101(get_task)
      166    0.001    0.000    0.008    0.000 DataProcess.py:105(<listcomp>)
        7    0.000    0.000    0.001    0.000 DataProce

# Test for given instances 

In [3]:
data_path_2 = os.path.dirname(os.path.abspath('__file__')) + "/Instances/dense_10_30_3_1_I.full"
EVA_TREE,GRAPH,NB_NODES = read_data(data_path_2)
NB_EVA_NODES = len(EVA_TREE)
NB_EDGES = len(GRAPH)
LIST_EVA_NODES = [item[0] for item in EVA_TREE]

In [13]:
#print_data(data_path_2)

In [4]:
print('Borne inferieur = {} mins'.format(get_borne_inf(LIST_EVA_NODES,EVA_TREE,GRAPH)))

Borne inferieur = 91 mins


In [6]:
endtime,init_sol= get_borne_sup(EVA_TREE,GRAPH)
print('Borne superieur = {} mins'.format(endtime))
init_sol

Borne superieur = 184 mins


[[279, 71, 0],
 [163, 70, 0],
 [384, 71, 56],
 [227, 71, 100],
 [359, 70, 120],
 [285, 71, 0],
 [275, 70, 54],
 [268, 132, 80],
 [344, 71, 11],
 [323, 71, 100]]

In [7]:
cp = cProfile.Profile()
cp.enable()
endtime,best_solution= LocalSearchRun(init_sol,EVA_TREE,GRAPH,n_iter=10)
cp.disable()
cp.print_stats()
print('solution = ',best_solution)
print('end time = ',endtime)

Iteration 0:
[279, 163, 285, 344, 275, 384, 268, 227, 323, 359]  =>  184
Iteration 1:
[227 163 285 344 275 384 268 279 323 359]  =>  181
Iteration 2:
[285 163 227 344 275 384 268 279 323 359]  =>  180
Iteration 3:
[285 163 227 344 275 384 268 279 323 359]  =>  180
Iteration 4:
[285 163 227 344 275 384 268 279 323 359]  =>  180
Iteration 5:
[285 163 227 344 275 384 268 279 323 359]  =>  180
Iteration 6:
[285 163 227 344 275 384 268 279 323 359]  =>  180
         3114559 function calls (3114523 primitive calls) in 62.729 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   62.729   62.729 <ipython-input-7-a1c5c073a1a3>:3(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-7-a1c5c073a1a3>:4(<module>)
    11135    0.155    0.000   13.465    0.001 DataProcess.py:101(get_task)
    11135    0.290    0.000   12.731    0.001 DataProcess.py:105(<listcomp>)
      148    0.006    0.000    1.4

In [19]:
cp = cProfile.Profile()
cp.enable()
endtime,best_solution,algo,exc = LocalSearchRandomStart(EVA_TREE,GRAPH,n_iter=10,n_start_points=5)
create_solution_file('dense_10_30_3_1_I',best_solution,endtime,algo,exc)
cp.disable()
cp.print_stats()
print('solution = ',best_solution)
print('end time = ',endtime)

# Test with due_date problem

In [ ]:
# We are thinking about it....